 **IBM Virus Species Jump Hackathon at Deep Learning IndabaX 2019 in Durban**

Goal: Predict whether an unknown animal virus could potentially infect humans given its genome (DNA or RNA) sequence.

Importance:

Many old and new dangerous viruses infecting humans emerge from animals. Emergence of new viruses is currently considered one of the biggest existential threats facing humanity. In 1918, a world-wide flu outbreak killed over 50 million people around the world. Ebola virus is thought to have come from bats, HIV virus is thought to have come from monkeys, SARS virus potentially came from birds, and many more. We can today sequence thousands of viruses in animals but it is hard to identify which of the tens of thousands of viruses present in wild and domestic animals could potentially cross over to infect humans. Thus, a computational model to predict whether an animal virus can infect humans would be of huge importance.

Training Data: Genome sequences of 70 human viruses that can be easily transmitted from animals to humans, e.g. ebola (Class Label: Zoonotic) and sequences of another 70 viruses that cannot be easily transmitted from animals to humans (Class Label: Non-Zoonotic).

The genome sequences are basically a string of 4 characters (AGCT) and the sequence of each virus ranges from 2000 letters to 10,000 characters. If you feel the training/test dataset is small, an option is to fragment each virus genome into smaller pieces which could easily create a training set of 5,000 to 10,000 samples in each class (i.e we can fragment each viral genome into 100 pieces).

Test Data: 60 Genome sequences with the class label hidden from participants but provided to hackathon organizers (30 from each class).

Evaluation: Participants will have to provide a deep learning model and their predictions for each provided test sample. Evaluation will be based on the follwing two criteria:

    Most innovative model
    Performance based on AUC/ precision-recall curves

Let's get started. Import all the necessary Python libraries.

In [16]:
import numpy as np
import pandas as pd
import string
import re
import matplotlib

Load the data uploaded to the current Colab instance under /content/sample_data/

In [17]:
#with open("./content/NonZoonoticVirusesTrain.fasta") as f:
with open("./content/NonZoonoticVirusesStringent0_0_GlobalViromeProject.fasta") as f:
    NonZoo_raw_data  = f.read()
    
#with open("./content/ZoonoticVirusesTrain.fasta") as g:
with open("./content/ZoonoticVirusSequencesStringentGlobalViromeProject.fasta") as g:
    Zoo_raw_data  = g.read()
    
with open("./content/VirusesTestInput.fasta") as h:
    test_raw_data  = h.read()

Preprocessing the data.

In [18]:
NonZoodata = NonZoo_raw_data.split(">")
Zoodata = Zoo_raw_data.split(">")
testdata = test_raw_data.split(">")

print(NonZoodata)
print(Zoodata)
print(testdata)

# dump the epmpy string in position [0]
NonZoodata2 = NonZoodata[1:]
Zoodata2 = Zoodata[1:]
testdata2 = testdata[1:]

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Separate names and sequences into two lists.

In [37]:
NonZoo_new_data = [x.split("\n",1) for x in NonZoodata2]
Zoo_new_data = [x.split("\n",1) for x in Zoodata2]
test_new_data = [x.split("\n",1) for x in testdata2]

NonZoo_id_name = []
Zoo_id_name = []
test_id_name = []
NonZoo_genome_sequence = []
Zoo_genome_sequence = []
test_genome_sequence = []

for x in NonZoo_new_data:
    NonZoo_id_name.append(x[0])
    NonZoo_genome_sequence.append(x[1])
    
for x in Zoo_new_data:
    Zoo_id_name.append(x[0])
    Zoo_genome_sequence.append(x[1])
    
for x in test_new_data:
    test_id_name.append(x[0])
    test_genome_sequence.append(x[1])

Separate into ID, Description and Sequence. From the data, you can just plit the ID using the first occurance of a "white space".

In [38]:
def clean_id_names(text):
    id_and_names = text.split(' ',1)
    return id_and_names

In [39]:
ZooID_Description = []
NonZooID_Description = []
testID_Description = []
for x in Zoo_id_name:
    ZooID_Description.append(clean_id_names(x))
for x in NonZoo_id_name:
    NonZooID_Description.append(clean_id_names(x)) 
for x in test_id_name:
    testID_Description.append(clean_id_names(x))

In [40]:
Zoo_ID = []
Zoo_Description = []

for x in ZooID_Description:
    Zoo_ID.append(x[0])
    Zoo_Description.append(x[1])
    
NonZoo_ID = []
NonZoo_Description = []

for x in NonZooID_Description:
    NonZoo_ID.append(x[0])
    NonZoo_Description.append(x[1])
    
test_ID = []
test_Description = []

for x in testID_Description:
    test_ID.append(x[0])
    test_Description.append(x[1])

Clean the sequences.

In [41]:
def clean_text(text):
    
    remove_lower = lambda text: re.sub('[a-z]', '', text)
    
    text = remove_lower(text)
    text = text.strip()
    text = text.replace('\n', '')
    return text

In [42]:
NonZoo_clean_sequences = []
Zoo_clean_sequences = []
test_clean_sequences = []
for seq in NonZoo_genome_sequence:
    NonZoo_clean_sequences.append(clean_text(seq))
    
for seq in Zoo_genome_sequence:
    Zoo_clean_sequences.append(clean_text(seq))
    
for seq in test_genome_sequence:
    test_clean_sequences.append(clean_text(seq))

Now we create two lists for classification, zeros and ones.

In [43]:
def zerolistmaker(n):
    listofzeros = [0] * n
    return listofzeros

def onelistmaker(n):
    listofzeros = [1] * n
    return listofzeros

In [44]:
Zoolabels = zerolistmaker(len(Zoo_clean_sequences)) #Zootonic viruses as Class 0.
NonZoolabels = onelistmaker(len(NonZoo_clean_sequences)) #NonZootonic viruses as Class 1.

Now merge the data and the labels.

In [45]:
Zoo_data_frame = [list(x) for x in zip(Zoo_ID,Zoo_Description,Zoo_clean_sequences,Zoolabels)]
NonZoo_data_frame = [list(x) for x in zip(NonZoo_ID,NonZoo_Description,NonZoo_clean_sequences,NonZoolabels)]
test_data_frame = [list(x) for x in zip(test_ID,test_Description,test_clean_sequences)]
trainlist = Zoo_data_frame + NonZoo_data_frame
dataframe = pd.DataFrame(trainlist, columns = ['ID' , 'Description', 'Sequences', 'Labels'])
dataframe.head()

,ID,Description,Sequences,Labels
0,NC_003466.1,"Andes virus segment S, complete sequence",TAGTAGTAGACTCCTTGAGAAGCTACTGCTGCGAAAGCTGGAATGA...,0
1,NC_003468.2,"Andes virus segment L, complete genome",TAGTAGTAGACTCCGGGATAGAAAAAGTTAGAAAAATGGAAAAGTA...,0
2,NC_003467.2,"Andes virus segment M, complete genome",TAGTAGTAGACTCCGCAAGAAGAAGCAAAAAATTAAAGAAGTGAGT...,0
3,NC_009026.2,"Bussuquara virus, complete genome",AGTATTTCTTCTGCGTGAGACCATTGCGACAGTTCGTACCGGTGAG...,0
4,NC_004211.1,"Banna virus strain JKT-6423 segment 1, complet...",GTATTAAAAATTATCAACAAGGAATGGACATTCAAGAACAATTTGA...,0


In [46]:
testframe = pd.DataFrame(test_data_frame, columns = ['ID' , 'Description', 'Sequences'])
testframe.head()

,ID,Description,Sequences
0,NC_002728.1,"Nipah virus, complete genome",ACCAAACAAGGGAGAATATGGATACGTTAAAATATATAACGTATTT...
1,NC_005283.1,"Dolphin morbillivirus, complete genome",ACCAGACAAAGCTGGCTAGGGGTAGAATAACAGATAATGATAAATT...
2,NC_001498.1,"Measles virus, complete genome",ACCAAACAAAGTTGGGTAAGGATAGATCAATCAATGATCATATTCT...
3,NC_004148.2,"Human metapneumovirus, complete genome",ACGCGAAAAAAACGCGTATAAATTAAGTTACAAAAAAACATGGGAC...
4,NC_003266.2,"Human adenovirus E, complete genome",CATCATCAATAATATACCTTATTTTTTTTGTGTGAGTTAATATGCA...


In [47]:
# Check for any nulls values
print(dataframe.isnull().sum())
print(testframe.isnull().sum())

ID             0
Description    0
Sequences      0
Labels         0
dtype: int64
ID             0
Description    0
Sequences      0
dtype: int64


In [48]:
# remove unwanted features 
clean_data = dataframe.drop(['ID', 'Description'],1)
clean_data.reset_index(drop=True)

,Sequences,Labels
0,TAGTAGTAGACTCCTTGAGAAGCTACTGCTGCGAAAGCTGGAATGA...,0
1,TAGTAGTAGACTCCGGGATAGAAAAAGTTAGAAAAATGGAAAAGTA...,0
2,TAGTAGTAGACTCCGCAAGAAGAAGCAAAAAATTAAAGAAGTGAGT...,0
3,AGTATTTCTTCTGCGTGAGACCATTGCGACAGTTCGTACCGGTGAG...,0
4,GTATTAAAAATTATCAACAAGGAATGGACATTCAAGAACAATTTGA...,0
5,GTATAAATTTTGTATCCGATAGTAGCACATAGAATCACAATCACAT...,0
6,GTATTAAATTTTTGTAAGTCGTTATGGAATTATTTAGTGACAGTGG...,0
7,GTATTTAAAATTCATGTTTTTGCATCATGGCGTGGGTTACGCAAGC...,0
8,GTATTAAAAATTACAAGAACCTAACATTGCAATGGAGATCTTGAGA...,0
9,GTATTTAAAATTATAGAAAGTTCTGAACCTAGGGGTCTTTCTGTCT...,0


In [49]:
# remove unwanted features 
clean_test = testframe.drop(['ID', 'Description'],1)
clean_test.reset_index(drop=True)

,Sequences
0,ACCAAACAAGGGAGAATATGGATACGTTAAAATATATAACGTATTT...
1,ACCAGACAAAGCTGGCTAGGGGTAGAATAACAGATAATGATAAATT...
2,ACCAAACAAAGTTGGGTAAGGATAGATCAATCAATGATCATATTCT...
3,ACGCGAAAAAAACGCGTATAAATTAAGTTACAAAAAAACATGGGAC...
4,CATCATCAATAATATACCTTATTTTTTTTGTGTGAGTTAATATGCA...
5,TTAAAACAGCCTGTGGGTTGTTCCCACCCACAGGCCCATTGGGCGC...
6,CGCACCGGGGATCCTAGGCAATTTGGTTGTCTTTTTTTGAGGCCTT...
7,GGCTCAAGCCTCTCGCGGCTGTGCAAGTGGAAAAAAAATTTTTTTA...
8,AAAAGGTGTGTTCTCTTATTGTAGCTAACAACAATCTTACTTACAG...
9,CGCACCGGGGATCCTAGGCTTTTTGGATTGCGCTTTCCTCTAGATC...


# **This is where your model starts. Train your model on clean_data dataframe and predict classes for clean_test dataframe**

In [50]:
clean_data.head() # Training data

,Sequences,Labels
0,TAGTAGTAGACTCCTTGAGAAGCTACTGCTGCGAAAGCTGGAATGA...,0
1,TAGTAGTAGACTCCGGGATAGAAAAAGTTAGAAAAATGGAAAAGTA...,0
2,TAGTAGTAGACTCCGCAAGAAGAAGCAAAAAATTAAAGAAGTGAGT...,0
3,AGTATTTCTTCTGCGTGAGACCATTGCGACAGTTCGTACCGGTGAG...,0
4,GTATTAAAAATTATCAACAAGGAATGGACATTCAAGAACAATTTGA...,0


In [51]:
clean_test.head()

,Sequences
0,ACCAAACAAGGGAGAATATGGATACGTTAAAATATATAACGTATTT...
1,ACCAGACAAAGCTGGCTAGGGGTAGAATAACAGATAATGATAAATT...
2,ACCAAACAAAGTTGGGTAAGGATAGATCAATCAATGATCATATTCT...
3,ACGCGAAAAAAACGCGTATAAATTAAGTTACAAAAAAACATGGGAC...
4,CATCATCAATAATATACCTTATTTTTTTTGTGTGAGTTAATATGCA...


# **Make predictions on test data and save to file for submission**

In [53]:
#clean_data = clean_data.drop(109)
#clean_data = clean_data.reset_index(drop=True)

In [71]:
#clean_data['Sequences'][16]= clean_data['Sequences'][16][0:len(clean_data['Sequences'][16])-15]
#clean_data['Sequences'][53]= (clean_data['Sequences'][53])[17:]
#clean_data['Sequences'][54]= (clean_data['Sequences'][54])[16:]
#clean_data['Sequences'][55]= (clean_data['Sequences'][55])[17:]

def rotate(strg, n):
    return strg[n:] + strg[:n]

def feature_engineer(data):
  protein = {"TTT" : "F", "CTT" : "L", "ATT" : "I", "GTT" : "V",
             "TTC" : "F", "CTC" : "L", "ATC" : "I", "GTC" : "V",
             "TTA" : "L", "CTA" : "L", "ATA" : "I", "GTA" : "V",
             "TTG" : "L", "CTG" : "L", "ATG" : "M", "GTG" : "V",
             "TCT" : "S", "CCT" : "P", "ACT" : "T", "GCT" : "A",
             "TCC" : "S", "CCC" : "P", "ACC" : "T", "GCC" : "A",
             "TCA" : "S", "CCA" : "P", "ACA" : "T", "GCA" : "A",
             "TCG" : "S", "CCG" : "P", "ACG" : "T", "GCG" : "A",
             "TAT" : "Y", "CAT" : "H", "AAT" : "N", "GAT" : "D",
             "TAC" : "Y", "CAC" : "H", "AAC" : "N", "GAC" : "D",
             "TAA" : "Z", "CAA" : "Q", "AAA" : "K", "GAA" : "E",
             "TAG" : "Z", "CAG" : "Q", "AAG" : "K", "GAG" : "E",
             "TGT" : "C", "CGT" : "R", "AGT" : "S", "GGT" : "G",
             "TGC" : "C", "CGC" : "R", "AGC" : "S", "GGC" : "G",
             "TGA" : "Z", "CGA" : "R", "AGA" : "R", "GGA" : "G",
             "TGG" : "W", "CGG" : "R", "AGG" : "R", "GGG" : "G" 
             }



  Features= []
  counter=0 
  for item in data['Sequences']:
    #Basline DNA Sequencing
    #print(counter)
    counter+=1
    DNA_length= len(item)
    A=item.count("A")/DNA_length
    C=item.count("C")/DNA_length
    G=item.count("G")/DNA_length
    T=item.count("T")/DNA_length


    protein_length = DNA_length//3
    #Protein Sequence/BOW
    protein_sequence = ""
    for i in range(0, len(item)-(3+len(item)%3), 3):
      #if protein[item[i:i+3]] == "Z" :
      #    break
      
      if item[i:i+3] in protein:
        protein_sequence += protein[item[i:i+3]]
      
      

    F=protein_sequence.count("F")/protein_length
    L=protein_sequence.count("L")/protein_length
    I=protein_sequence.count("I")/protein_length
    M=protein_sequence.count("M")/protein_length
    V=protein_sequence.count("V")/protein_length
    S=protein_sequence.count("S")/protein_length
    P=protein_sequence.count("P")/protein_length
    T=protein_sequence.count("T")/protein_length
    A=protein_sequence.count("A")/protein_length
    Y=protein_sequence.count("Y")/protein_length
    H=protein_sequence.count("H")/protein_length
    N=protein_sequence.count("N")/protein_length
    D=protein_sequence.count("D")/protein_length
    E=protein_sequence.count("E")/protein_length
    G=protein_sequence.count("G")/protein_length
    R=protein_sequence.count("R")/protein_length
    Q=protein_sequence.count("Q")/protein_length
    K=protein_sequence.count("K")/protein_length
    W=protein_sequence.count("W")/protein_length
    Z=protein_sequence.count("Z")/protein_length
    Features.append([A,C,G,T,DNA_length,F,L,I,M,V,S,P,T,A,Y,H,N,D,E,G,R,Q,K,W,Z])
  return Features
  
  
X = np.array(feature_engineer(clean_data) )   
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
#X = np.array(Features)
y = clean_data['Labels'].values
#print(X.shape)
clf = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=0)
clf.fit(X, y)
pred = clf.predict(X)
accuracy = (y==pred).sum() / len(y)
#print(accuracy)


###Validation Data Cleaning
X_FINAL_TEST = np.array(feature_engineer(clean_test))

In [56]:
# latest version crashes on fit()
!pip install xgboost==0.80  

In [78]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import numpy as np

# ignore deprecation warnings
import warnings
with warnings.catch_warnings():
    warnings.filterwarnings("ignore",category=DeprecationWarning)

    test_acc = 0
    rep = 1000
    test_accs = []
    for i in range(rep):
      #X, y = shuffle(X, y)
      X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

      clf =XGBClassifier(max_depth=5,min_child_weight=3, gamma=0.2, subsample=0.6, colsample_bytree=1.0)#, n_estimators=50)#, learning_rate=0.1, n_estimators=100)
      clf.fit(X_train, y_train)
      pred = clf.predict(X_train)
      accuracy = (y_train==pred).sum() / len(y_train)
    #   print("Train:",accuracy)

      pred = clf.predict(X_test)
      accuracy = (y_test==pred).sum() / len(y_test)
    #   print("Test:",accuracy)
      test_accs.append(accuracy)
    test_accs = np.array(test_accs)
    print(test_accs.mean(),"+/-",test_accs.std())
    print("Min:",test_accs.min(),"Max:",test_accs.max())

0.5791904761904761 +/- 0.10128909480108697
Min: 0.23809523809523808 Max: 0.9047619047619048


In [37]:
clf = XGBClassifier(max_depth=5,min_child_weight=3, gamma=0.2, subsample=0.6, colsample_bytree=1.0)
clf.fit(X, y)
pred = clf.predict(X)
train_accuracy = (y==pred).sum() / len(y)
print("Train:",train_accuracy)
y_test_pred = clf.predict(X_FINAL_TEST)
print(y_test_pred)
print(len(y_test_pred))

Train: 0.945273631840796
[0 1 0 0 0 0 0 1 1 0 1 0 0 0 1 0 0 1 0 1 1 1 1 1 1 0 0 1 1 1 1 1 0 0 1 0 0
 0 0 1 0 1 0 1 0 0 0 1 1 0 1 1 0 0 0 0 1 1 1 0]
60


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [ ]:
# y_pred = [str(x) for x in y_test_pred]
# pred_y = ","
# pred_y = pred_y.join(y_pred)
# with open("./content/VirusesTestPrediction.fasta","w+") as o:
#     o.write(pred_y)